In [1]:
import spconv.pytorch as spconv
import torch
from torch import nn

In [2]:
spconv3d = spconv.SparseConv3d(
    in_channels=5, 
    out_channels=16, 
    kernel_size=3, 
    stride=1, 
    padding=0,
    bias=False
)

In [4]:
spconv3d.weight.shape

torch.Size([16, 3, 3, 3, 5])

In [5]:
conv3d = nn.Conv3d(
    in_channels=5, 
    out_channels=16, 
    kernel_size=3, 
    stride=1, 
    padding=0,
    bias=False
)

In [6]:
conv3d.weight.requires_grad_(False)

Parameter containing:
tensor([[[[[-2.9906e-02,  4.1391e-02, -2.4905e-03],
           [ 6.8564e-02, -1.8872e-02, -1.1939e-02],
           [-3.8255e-03,  1.2062e-03,  9.0661e-03]],

          [[-4.1008e-02,  1.0133e-02,  8.5754e-02],
           [-4.0818e-02, -3.1230e-02, -4.6893e-02],
           [-3.7212e-02,  3.8643e-02,  3.6913e-02]],

          [[-6.0373e-02, -8.2668e-02,  6.0592e-04],
           [ 7.1545e-02,  6.8577e-02,  7.5498e-02],
           [ 7.2541e-02, -8.1798e-04, -2.4814e-02]]],


         [[[ 6.2085e-02,  7.0087e-02,  6.4437e-02],
           [ 7.1197e-02, -7.3215e-02, -4.1082e-02],
           [-6.1871e-02,  1.6108e-02, -7.9481e-02]],

          [[-2.1665e-02, -5.1385e-03,  1.9696e-02],
           [ 4.1940e-02,  6.7375e-02,  1.6773e-02],
           [ 4.4584e-02, -2.8041e-02, -6.5841e-02]],

          [[ 2.4576e-02, -3.9630e-02, -7.4896e-02],
           [ 7.3230e-02,  7.8814e-02,  2.9902e-02],
           [-2.2057e-02, -1.1097e-02, -6.0186e-04]]],


         [[[ 7.7484e-02, -

In [7]:
conv3d.weight.shape

torch.Size([16, 5, 3, 3, 3])

In [8]:
spconv3d_weights = torch.randn(16, 3, 3, 3, 5)

In [9]:
reshaped_weights = spconv3d_weights.permute(0, 4, 1, 2, 3)

In [10]:
conv3d.weight.data = reshaped_weights

In [11]:
spconv3d.weight.data = spconv3d_weights

In [12]:
# Parameters
N = 10  # number of non-zero points
num_channels = 5  # number of feature channels per point
ndim = 3  # number of spatial dimensions (e.g., 3D convolution)
batch_size = 2  # number of batches

# Generating random features
features = torch.randn(N, num_channels).to('cuda')

# Generating random indices for a 3D sparse tensor with batch indices
spatial_shape = (100, 100, 100)  # Define the spatial shape of the sparse tensor
indices = torch.zeros(N, ndim + 1, dtype=torch.int32).to('cuda')
indices[:, 0] = torch.randint(0, batch_size, (N,))  # Batch indices
for i in range(1, ndim + 1):
    indices[:, i] = torch.randint(0, spatial_shape[i - 1], (N,))  # Spatial indices

# Creating the SparseConvTensor
x = spconv.SparseConvTensor(features, indices, spatial_shape, batch_size)

# Converting sparse tensor to dense NCHW tensor (just for visualization; might be memory intensive)
x_dense_NCHW = x.dense()
print(x_dense_NCHW.shape)  # Should match the expected dense shape [batch_size, num_channels, *spatial_shape]


torch.Size([2, 5, 100, 100, 100])


In [13]:
conv3d.to('cuda')

Conv3d(5, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)

In [14]:
spconv3d.to('cuda')

SparseConv3d(5, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[0, 0, 0], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)

In [15]:
spconv_y = spconv3d(x)
y = conv3d(x_dense_NCHW)

In [16]:
spconv_y = spconv_y.dense()

In [18]:
spconv_y.shape

torch.Size([2, 16, 98, 98, 98])

In [17]:
y.shape

torch.Size([2, 16, 98, 98, 98])

In [20]:
torch.allclose(spconv_y, y)

False